##### Imports

In [ ]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from scr.feature_engineering import *
from scr.rnn import *
from scr.model_checkpoint_manager import *
from scr.utils import *
from scr.game import  *

##### Model reading with best hypers

In [ ]:
import yaml
from pathlib import Path

# Assuming models_dir is a Path object pointing to your main models directory
models_dir = Path('models')  # Replace with actual path

# Paths
best_model_dir = models_dir / 'best_model'
config_file_path = best_model_dir / 'config.yml'

# Load the configuration
with open(config_file_path, 'r') as file:
    loaded_config = yaml.safe_load(file)

In [ ]:
# Initialize the model
# Assuming RNN is your model class and loaded_config is the configuration dictionary
model = RNN(loaded_config)
# Path to the saved model state
final_model_path = best_model_dir / 'final_model.pth'

# Load the model's state dictionary
model.load_state_dict(torch.load(final_model_path)) # , map_location=torch.device('cuda')))
# model.eval()  # Set the model to evaluation mode

##### Reading data for testing

In [ ]:
# Limit the number of words to a smaller number for debugging
train = word_list = read_words('/home/sayem/Desktop/Hangman/words_250000_train.txt', limit=None)

test = unseen_words = read_words('/home/sayem/Desktop/Hangman/20k.txt', limit=None)

##### Playing games

In [ ]:
# Import the necessary functions
from scr.feature_engineering import \
    process_single_word, get_missed_characters
    
from scr.game import predict_next_character, simulate_game
from scr.rnn import RNN
import random

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

char_frequency = calculate_char_frequencies(word_list) # dont change
max_word_length = max(len(word) for word in word_list) # dont change

model = model.to(device)

def play_multiple_games(model, num_games, word_list, \
    char_to_idx, idx_to_char, char_frequency, max_word_length, device):
    game_results = []
    for _ in range(num_games):
        random_word = random.choice(word_list)
        with torch.no_grad():
            won, final_word, attempts_used = simulate_game(
                model, 
                random_word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device,
                if_init_guess=False,
                normalize=True, 
                max_attempts=6
            )
        game_results.append((won, final_word, attempts_used))
    return game_results

num_games = 1000

results = play_multiple_games(model, num_games, \
    unseen_words, char_to_idx, idx_to_char, \
        char_frequency, max_word_length, device)

# Analyzing results
total_wins = sum(result[0] for result in results)

( total_wins / num_games) * 100

In [ ]:
get_missed_characters('___', set(), char_to_idx)